### `Importing Libiraries`

In [2]:
import numpy as np
import pandas as pd
import os
import re
import joblib 
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from nltk.corpus import stopwords
from sklearn.svm import SVC
from textblob import TextBlob
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report

#### `Reading The Dataset seintment 140`

In [3]:
path = os.path.dirname(os.getcwd())
print(path)

df = pd.read_csv(os.path.join(path,'data/tweets.csv'), encoding='latin1')

df.columns = ['target', 'id', 'date', 'query_string', 'user', 'text']

e:\DEPO\Seintment Analysis


#### `EDA`

In [24]:
df.head()

,target,id,date,query_string,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [25]:
df.describe()

,target,id
count,1.599999e+06,1.599999e+06
mean,2.000001e+00,1.998818e+09
std,2.000001e+00,1.935757e+08
min,0.000000e+00,1.467811e+09
25%,0.000000e+00,1.956916e+09
50%,4.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   target        1599999 non-null  int64 
 1   id            1599999 non-null  int64 
 2   date          1599999 non-null  object
 3   query_string  1599999 non-null  object
 4   user          1599999 non-null  object
 5   text          1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [27]:
df.target.value_counts()

target
4    800000
0    799999
Name: count, dtype: int64

#### `Data Preprocessing`

In [28]:
df.drop(['id', 'date', 'query_string', 'user'], axis=1, inplace=True)
df.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [29]:
df['target'] = df['target'].map({0: 0, 4: 1})
df.head()

,target,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [30]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'@\w+', '', text)              # remove mentions
    text = re.sub(r'[^\w\s]', '', text)           # remove punctuation
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(clean_text)
df.drop('text', axis=1, inplace=True)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,target,clean_text
0,0,upset cant update facebook texting might cry r...
1,0,dived many times ball managed save 50 rest go ...
2,0,whole body feels itchy like fire
3,0,behaving im mad cant see
4,0,whole crew


#### `1st Using Original Logistic Regression`

In [31]:
X = df['clean_text']
y = df['target']

tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_features=10000)
X_vec = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)


In [32]:
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs'],
    'penalty': ['l2'],
    'max_iter': [100, 200, 500, 1000],
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best Params:", grid.best_params_)

# Evaluate
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

e:\DEPO\Seintment Analysis\saenv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Best Params: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

           0       0.79      0.75      0.77    159494
           1       0.76      0.80      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.78      0.77      0.77    320000
weighted avg       0.78      0.77      0.77    320000



In [34]:
joblib.dump(grid.best_estimator_, os.path.join(path, 'models/logistic_model.pkl'))

# Save vectorizer
joblib.dump(tfidf, os.path.join(path, 'models/tfidf_vectorizer.pkl'))

['e:\\DEPO\\Seintment Analysis\\models/tfidf_vectorizer.pkl']